# POC SNE - Atendimento Serpro

## Instalação de pacotes necessários

In [1]:
# Importações básicas - Descomentar as linhas abaixo:
#!pip install pandas
#!pip install tensorflow
#!pip install keras
#!pip install scikit-learn
#!pip install nltk
#!pip install ezodf
#!pip install lxml
#import nltk
#nltk.download('all')
#!pip install spacy
#!python -m spacy download pt_core_news_sm

## Importações e criação de funções

In [2]:
import ezodf
import pandas as pd
import sys
import unicodedata
import re
import time
import spacy
import pickle
from unicodedata import normalize
from nltk.util import ngrams
from nltk import tokenize # Tokenização
from nltk.corpus import stopwords # Stopwords
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import RSLPStemmer
# Função para mostrar as planilhas do documento
def show_ods_sheets(filename):
    doc = ezodf.opendoc(filename)
    print("Spreadsheet contains %d sheet(s)." % len(doc.sheets))
    for sheet in doc.sheets:
        print("-"*40)
        print("   Sheet name : '%s'" % sheet.name)
        print("Size of Sheet : (rows=%d, cols=%d)" % (sheet.nrows(), sheet.ncols()) )
# Função para converter ODS em DataFrame do Pandas
def read_sheet_ods_as_panda_dataframe(filename, sheet_no=0, header=0):
    tab = ezodf.opendoc(filename=filename).sheets[sheet_no]
    return pd.DataFrame({col[header].value:[x.value for x in col[header+1:]]
                         for col in tab.columns()})

## Abrindo o arquivo como um DataFrame

In [3]:
# Exibindo informações sobre o arquivo ODS
FILENAME = '/home/03662232677/workspace-neon/PUCOMEX/SS_SNE.ods'
show_ods_sheets(FILENAME)

Spreadsheet contains 1 sheet(s).
----------------------------------------
   Sheet name : 'Incidentes'
Size of Sheet : (rows=34120, cols=15)


In [4]:
# Lendo o arquivo ODS com o Pandas
df = read_sheet_ods_as_panda_dataframe(FILENAME)

## Inspecionando e trabalhando o DataFrame

In [5]:
# Mostrando os primeiros registros
df.head()

,ticketid,ownergroup,description,externalsystem,status,classificationid,serpronivelaten,descricaodemanda,solution,classificationid1,description2,relatedreckey,internalpriority,solucaoparausuario,None
0,2018SS/0000401275,ATCDMGSNE,SNE,EMAIL,CLOSED,10252,1NIVEL,Solicitação de serviço: 2018SS/0000401273 Qual...,,10252,SNE,,9,"Prezado Senhor André, O Sistema de Notificação...",None
1,2017SS/0000183234,UNCEDFSNE,SNE,EMAIL,CLOSED,10252,3NIVEL,Usuário informa que recebeu umas notificações ...,,10252,SNE,,9,"Sr. Usuário,\n\nO órgão autuador faz algumas v...",None
2,2017SS/0000183525,CAGSBRSNE,SNE,EMAIL,CLOSED,10252,2NIVEL,Usuário informa que emitiu um boleto que vence...,,10252,SNE,,9,"Prezado Angelo,\n\nQual a data de vencimento d...",None
3,2017SS/0000184325,CAGSBRSNE,SNE,EMAIL,CLOSED,10252,2NIVEL,Usuário informa que esta tentando cadastrar um...,,10252,SNE,,9,"Prezado Tiago,\nInformamos que o código de seg...",None
4,2017SS/0000242075,CAGSBRSNE,SNE,EMAIL,CLOSED,10252,2NIVEL,Assunto: MULTA PAGA VOLTANDO A CONTAR NO SISTE...,2016SOL/00400157,10252,SNE,,9,"Prezado Jeferson,\n\n\nFavor nos enviar PLACA ...",None


In [6]:
# Selecionando apenas as colunas desejadas
df = df[['description','ownergroup','descricaodemanda','solution','solucaoparausuario']]

In [7]:
# Mudando o nome das colunas
df.columns = ['descricao','grupo','descricao_demanda','solucao','solucao_usuario']
df.head()

,descricao,grupo,descricao_demanda,solucao,solucao_usuario
0,SNE,ATCDMGSNE,Solicitação de serviço: 2018SS/0000401273 Qual...,,"Prezado Senhor André, O Sistema de Notificação..."
1,SNE,UNCEDFSNE,Usuário informa que recebeu umas notificações ...,,"Sr. Usuário,\n\nO órgão autuador faz algumas v..."
2,SNE,CAGSBRSNE,Usuário informa que emitiu um boleto que vence...,,"Prezado Angelo,\n\nQual a data de vencimento d..."
3,SNE,CAGSBRSNE,Usuário informa que esta tentando cadastrar um...,,"Prezado Tiago,\nInformamos que o código de seg..."
4,SNE,CAGSBRSNE,Assunto: MULTA PAGA VOLTANDO A CONTAR NO SISTE...,2016SOL/00400157,"Prezado Jeferson,\n\n\nFavor nos enviar PLACA ..."


In [8]:
# Exibindo estatísticas do pandas
df.describe()

,descricao,grupo,descricao_demanda,solucao,solucao_usuario
count,34117,34117,34117,34117,34117
unique,49,23,32139,89,12451
top,"SNE,INFRAÇÃO / MULTA NÃO DISPONÍVEL NO SNE",ATCDMGSNE,,2017SOL/0000002593,"Prezado(a) Sr(a),\n\no cadastro pode ser feito..."
freq,5467,10463,161,5739,1119


In [9]:
# Removendo valores vazios em descricao_demanda
df = df.dropna(subset=['descricao_demanda']) 
df = df[df.descricao_demanda != '']
df.describe()

,descricao,grupo,descricao_demanda,solucao,solucao_usuario
count,33956,33956,33956,33956,33956
unique,48,23,32138,88,12367
top,"SNE,INFRAÇÃO / MULTA NÃO DISPONÍVEL NO SNE",ATCDMGSNE,Adesão ao SNE,2017SOL/0000002593,"Prezado(a) Sr(a),\n\no cadastro pode ser feito..."
freq,5447,10452,145,5686,1110


In [10]:
# removendo colunas indesejadas de grupo
df = df.dropna(subset=['grupo']) 
df = df[df.grupo != '']
grupos = df['grupo'].unique()
print(len(grupos))
grupos

22


array(['ATCDMGSNE', 'UNCEDFSNE', 'CAGSBRSNE', 'GSGSSPCENTRODECOMANDO',
       'ATGSRJ', 'ATGSPEDENATRAN', 'ATGSBRTERC', 'CACDBRLOJASSNE',
       'GSCDSPLINUX', 'CACDBRSNE', 'GSDEBAAPPSNE', 'GSCDSP',
       'GSGSDFCENTRODECOMANDO', 'ATGSMG', 'CAGSBRRENAVAM', 'CACDBRRADAR',
       'CACDBRCNHDIGITAL', 'CAGSBRRENAJUD', 'ATGSRJELETR', 'ATGSBREMAIL',
       'ATGSPE', 'GSSSDFRADAR'], dtype=object)

In [11]:
# Removendo quebra de linha da descricao
df['descricao'] = df['descricao'].replace(r'\n','', regex=True)
descricoes = df['descricao'].unique()
print(len(descricoes))
descricoes

48


array(['SNE', 'SNE,ACESSO', 'SNE,ACESSO,SISTEMA INDISPONÍVEL-LENTIDÃO',
       'SNE,ACESSO,SITIO INDISPONÍVEL- LENTIDÃO',
       'SNE,ACESSO,DÚVIDA - INFORMAÇÃO',
       'SNE,ACESSO,NAVEGADOR DE INTERNET', 'SNE,APLICAÇÃO',
       'SNE,APLICAÇÃO,ESCLARECIMENTO', 'SNE,CADASTRO',
       'SNE,CADASTRO,ACESSO', 'SNE,CADASTRO,DÚVIDA – INFORMAÇÃO',
       'SNE,CADASTRO,VEÍCULO', 'SNE,SOLICITAÇÃO',
       'SNE,DÚVIDA - INFORMAÇÃO', 'SNE,SOLICITAÇÃO,DÚVIDA - INFORMAÇÃO',
       'SNE,SITUAÇÃO NÃO PREVISTA', 'SNE,RECLAMACAO',
       'SNE,ERRO NA GERAÇÃO DE LINHA DIGITÁVEL/BOLETO DO DETRAN-DF',
       'SNE,SNE - ERRO NA GERAÇÃO DA LINHA DIGITÁVEL',
       'SNE,SNE - ERRO NO CADASTRO DO USUÁRIO NO SNE',
       'SNE,SNE - ERRO NO CADASTRO DE VEÍCULO NO SNE',
       'SNE,SNE - CADASTRO PESSOA JURÍDICA - CNPJ',
       'SNE,SNE - CADASTRO DE USUÁRIO PESSOA FÍSICA',
       'SNE,INFRAÇÃO / MULTA NÃO DISPONÍVEL NO SNE',
       'SNE,BAIXA DA MULTA NÃO DISPONÍVEL NO SNE',
       'SNE,SNE - MSG ERRO: OCORREU

In [12]:
# Extraindo o segundo e terceiro termo após a vírgula
import numpy as np
df['sistema'] =  (np.where(df['descricao'].str.contains(','),
                  df['descricao'].str.split(',').str[1],
                  df['descricao']))
df['classificacao'] =  (np.where(df['descricao'].str.contains(','),
                  df['descricao'].str.split(',').str[2],
                  df['descricao']))
# Reorganizando a ordem das colunas
df = df[['sistema','classificacao','grupo', 'descricao_demanda','descricao','solucao','solucao_usuario']]
# Removendo valores nulos das colunas criadas
df = df.dropna(subset=['sistema','classificacao'])
df = df[df.sistema != '']
df = df[df.classificacao != '']
df.describe()

,sistema,classificacao,grupo,descricao_demanda,descricao,solucao,solucao_usuario
count,6291,6291,6291,6291,6291,6291,6291
unique,7,19,20,5914,20,62,2056
top,SOLICITAÇÃO,DÚVIDA - INFORMAÇÃO,ATGSBRTERC,Usuário pergunta como pode se CADASTRAR no SIS...,"SNE,SOLICITAÇÃO,DÚVIDA - INFORMAÇÃO",,A consulta dos órgãos que fizeram adesão ao SN...
freq,1789,2041,2432,35,1789,1521,311


In [13]:
# Verificando quantos registros existem por sistema
df.groupby('sistema').count()

,classificacao,grupo,descricao_demanda,descricao,solucao,solucao_usuario
sistema,,,,,,
ACESSO,420,420,420,420,420,420
APLICAÇÃO,1438,1438,1438,1438,1438,1438
CADASTRO,1060,1060,1060,1060,1060,1060
SNE,572,572,572,572,572,572
SNE - ATENDIMENTO LOJAS,915,915,915,915,915,915
SNE - MSG ERRO: OCORREU UM ERRO INESPERADO,97,97,97,97,97,97
SOLICITAÇÃO,1789,1789,1789,1789,1789,1789


In [14]:
# Escolhendo um sistema para a POC
#SISTEMA_SELECIONADO = 'DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO'
#df = df.loc[df['sistema'] == SISTEMA_SELECIONADO]
# Mostrando quandos existem por classificacao
df.groupby('classificacao').count()

,sistema,grupo,descricao_demanda,descricao,solucao,solucao_usuario
classificacao,,,,,,
RETORNO INVÁLIDO DO SERVIDOR,97,97,97,97,97,97
ACESSO,13,13,13,13,13,13
DÚVIDA - INFORMAÇÃO,2041,2041,2041,2041,2041,2041
DÚVIDA – INFORMAÇÃO,968,968,968,968,968,968
ESCLARECIMENTO,1438,1438,1438,1438,1438,1438
NAVEGADOR DE INTERNET,53,53,53,53,53,53
SISTEMA INDISPONÍVEL-LENTIDÃO,100,100,100,100,100,100
SITIO INDISPONÍVEL- LENTIDÃO,15,15,15,15,15,15
SNE,572,572,572,572,572,572


In [15]:
# Removendo classificaoes com menos que X excemplos
MINIMO_REGISTROS = 50
counts = df['sistema'].value_counts()
df = df[df['sistema'].isin(counts[counts > MINIMO_REGISTROS].index)]
df.groupby('sistema').count()

,classificacao,grupo,descricao_demanda,descricao,solucao,solucao_usuario
sistema,,,,,,
ACESSO,420,420,420,420,420,420
APLICAÇÃO,1438,1438,1438,1438,1438,1438
CADASTRO,1060,1060,1060,1060,1060,1060
SNE,572,572,572,572,572,572
SNE - ATENDIMENTO LOJAS,915,915,915,915,915,915
SNE - MSG ERRO: OCORREU UM ERRO INESPERADO,97,97,97,97,97,97
SOLICITAÇÃO,1789,1789,1789,1789,1789,1789


In [16]:
#df2 = df[df.classificacao == 'SITUAÇÃO NÃO PREVISTA']
#df = df[df.classificacao != 'SITUAÇÃO NÃO PREVISTA']
#df.groupby('classificacao').count()

In [17]:
#df2.groupby('classificacao').count()

In [18]:
targets = df['sistema'].unique()
print(len(targets))
targets

7


array(['SNE', 'ACESSO', 'APLICAÇÃO', 'CADASTRO', 'SOLICITAÇÃO',
       'SNE - MSG ERRO: OCORREU UM ERRO INESPERADO',
       'SNE - ATENDIMENTO LOJAS'], dtype=object)

In [19]:
df.head()

,sistema,classificacao,grupo,descricao_demanda,descricao,solucao,solucao_usuario
0,SNE,SNE,ATCDMGSNE,Solicitação de serviço: 2018SS/0000401273 Qual...,SNE,,"Prezado Senhor André, O Sistema de Notificação..."
1,SNE,SNE,UNCEDFSNE,Usuário informa que recebeu umas notificações ...,SNE,,"Sr. Usuário,\n\nO órgão autuador faz algumas v..."
2,SNE,SNE,CAGSBRSNE,Usuário informa que emitiu um boleto que vence...,SNE,,"Prezado Angelo,\n\nQual a data de vencimento d..."
3,SNE,SNE,CAGSBRSNE,Usuário informa que esta tentando cadastrar um...,SNE,,"Prezado Tiago,\nInformamos que o código de seg..."
4,SNE,SNE,CAGSBRSNE,Assunto: MULTA PAGA VOLTANDO A CONTAR NO SISTE...,SNE,2016SOL/00400157,"Prezado Jeferson,\n\n\nFavor nos enviar PLACA ..."


## Preparação dos dados

In [20]:
# Inicializa o stemmer
stemmer = LancasterStemmer() 
# Stop words para o Português
portuguese_stops = set(stopwords.words('portuguese'))
portuguese_stops.update('é')
# Lista de palavras para remover com regex
include_regex = [r"^[A-Za-zçãàáâéêíóôõúüÂÃÁÀÉÊÍÓÔÕÚÜÇ'º°ª\.]*$",r"^[0-9\-/\\\.]*$"]
#include_regex = [r"^[A-Za-zçãàáâéêíóôõúüÂÃÁÀÉÊÍÓÔÕÚÜÇ'º°ª\.]*$"]
LIMITE_TAMANHO_PALAVRA = 3
# Dicionátio que armazena as pontuações a serem removidas
tbl = {} 
for code in [i for i in range(sys.maxunicode) if unicodedata.category(chr(i)).startswith('P')]:
    tbl[code] = ' '
# Função para remover pontuações das sentenças
def remove_punctuation(text): 
    return text.translate(tbl)
# Função para salvar uma lista em um binário
def save_list_to_file(list_to_save,file_name):
    with open(file_name, 'wb') as f:
        pickle.dump(list_to_save, f)
# Função para realizar o pré-processamento dos conteúdos dos arquivos
def create_doc(txt,target): 
    docs = []
    ngrs = []
    ret = {}
    content = remove_punctuation(txt)
    words = tokenize.word_tokenize(content, language='portuguese')     
    # Remove stopwords
    words = [word for word in words if word.lower() not in portuguese_stops] 
    # Stem de cada palavra, converte para minúsculo 
    words = [stemmer.stem(w.lower()) for w in words] 
    # Remove palavras menores que um limite
    words = [word for word in words if len(word) > LIMITE_TAMANHO_PALAVRA] 
    # Remove regex words
    words_aux = []
    for word in words:
        add = False
        for regex_exp in include_regex:
            if re.compile(regex_exp).match(word):
                add = True
        if add:
            words_aux.append(word)
        #else:
        #    print ('removerndo regex: ',word)
    words = words_aux
    for ngr in ngrams(words,2):
        ngrs.append(ngr)
    ret['ngrams'] = ngrs
    # Busca resultados e adiciona nos docs
    cats = []
    cats.append(target)
    docs.append((ngrs, cats, txt))
    ret['docs'] = docs
    return ret    

In [23]:
count = 0
# Lista de todas as palavras de todos os documentos
all_ngrams = []
# Uma lista de tuplas com as palavras das sentenças e o nome da categoria
docs = [] 
ngr = []
LIMITE = -1
for row_index, row in df.iterrows():
    txt = row['descricao_demanda']
    target = row['sistema']
    count += 1
    ret = create_doc(txt,target)
    docs.extend(ret['docs'])
    all_ngrams.extend(ret['ngrams'])
    if count % 100 == 0:
        print (count,' linhas processadas. Último processado: ',txt)
    if LIMITE != -1 and count == LIMITE:
        break
print (count,' arquivos processados. Último processado: ',txt)
print('\nForam geradas ',len(docs),' combinações e ',len(all_ngrams),' palavras')
print ('\n',len(targets),'Targets: ',targets)
print ('\n',len(docs),'Docs (primeiro): ',docs[:1])
print ('\n',len(all_ngrams),' N-Grams: ',all_ngrams[:100])    

100  linhas processadas. Último processado:  SNE
200  linhas processadas. Último processado:  estou tentando acessar o SNE para a Empresa ( Pessoa Jurídica ) mas aparece 
um erro q não localiza a pagina. Este Software esta disponível para empresa 
ou somente para pessoa física?
300  linhas processadas. Último processado:  REF. AUTO NOTIFICAÇÃO E AUTUAÇÃO Nº 0042792538 E Nº 0042792617 PELA PRF 
INFORMA PARA PAGTO SEM RECORRER E 40% COMO FAÇO NÃO ESTAMOS CONSEGUINDO 
IMPRIMIR BOLETO, SERPRO ESTA FORA AR
400  linhas processadas. Último processado:  Usuário deseja informações de como cadastrar pessoa Jurídica no SNE
500  linhas processadas. Último processado:  Usuário deseja informações sobre cadastro de CNPJ no sne.
600  linhas processadas. Último processado:  Usuário informa que baixou o app e ao clicar no link para confirmar o 
cadastro o sistema informa que o site não é seguro.
700  linhas processadas. Último processado:  Boa tarde, estou com problemas para emitir o boleto para pagamen

4500  linhas processadas. Último processado:  Usuário solicita orientações para saber como proceder a respeito do 
aplicativo do SNE.
4600  linhas processadas. Último processado:   Usuária solicita informações para efetuar o pagamento da multa com 40% de 
desconto
4700  linhas processadas. Último processado:  Usuário gerou boleto com desconto, porém perdeu o prazo de pagamento. O 
mesmo solicita informações sobre como conseguiu segunda via do boleto com 
desconto
 
Placa: jgm7222
Orgão : detran- df  
data da autuação:--
auto de infração st00770344
 renavam:  00948498803

4800  linhas processadas. Último processado:   Usuário relata que os descontos de 40% na multa efetuada pelo DETRAN- SC  
não está apresentando no SNE
4900  linhas processadas. Último processado:  GOSTARIA DE ADERIR PARA OBTER O DESCONTO.
5000  linhas processadas. Último processado:  Como descadastrar um veículo... OIE 1204.
5100  linhas processadas. Último processado:  Não consigo gerar o boleto da infração nem recebi

6200  linhas processadas. Último processado:  Não aparece meu veículo....afirma que não tem carro no meu CPF no renavam. 
Ou seja, não deve pra nada
https://appstoreconnect.apple.com/WebObjects/iTunesConnect.woa/ra/ng/app/1170007091/activity/ios/ratingsResponses

6291  arquivos processados. Último processado:  Olá, Um usuário escreveu uma nova resenha sobre seu app. Você pode 
responder a ela no Play Console.

Nova resenha em 17 de fev de 2019, 13:48 (GMT) ★ o app falha toda vez que 
tento entrar Responder 
<https://play.google.com/apps/publish?account=7590937500426216625#ReviewDetailsPlace:p=br.gov.serpro.denatran.sne&reviewid=gp:AOqpTOEInmekYGUYopFzNbqfQmfItb64IUgn4BCuOeJFtETFxe0M4G6-B1aWzaaWiPtkl8MXxNztO_xGqxipqJA> 
Atenciosamente, *equipe do Google Play* 
<https://support.google.com/googleplay/android-developer/answer/138230>. 
<https://support.google.com/googleplay/android-developer/answer/113417>. 
<https://play.google.com/apps/publish?account=7590937500426216625#EmailPrefsPlace>

In [25]:
# Store data (serialize)
save_list_to_file(targets,'targets.pickle')
save_list_to_file(all_ngrams,'all_ngrams.pickle')
save_list_to_file(docs,'docs.pickle')
print ('Arquivos salvos com sucesso!')

Arquivos salvos com sucesso!


FIM